# Setup

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# Question 1

In [2]:
img = cv.imread('dog1.jfif', 0)

eq1 = cv.equalizeHist(img)
eq2 = cv.equalizeHist(eq1)

cv.imshow('result', np.hstack((img, eq1, eq2)))
cv.waitKey(0)
cv.destroyAllWindows()

Comment: There is no difference between first and second output image since it is already equalized. It can't be equalized anymore.

# Question 2

In [3]:
img = cv.imread('electronic.jfif', 0)

sobelx = cv.Sobel(img, cv.CV_64F, 1,0, ksize = 1)
sobely = cv.Sobel(img, cv.CV_64F, 0,1, ksize = 1)

grad_mag_L2 = cv.magnitude(sobelx,sobely)
grad_mag_L2 = cv.convertScaleAbs(grad_mag_L2)

cv.imshow("result", np.hstack((img,grad_mag_L2)))
cv.waitKey(0)
cv.destroyAllWindows()

Kernel size 1 is chosen as there is less white line (noises) in the images compared to kernel size 3.

In [4]:
blur = cv.GaussianBlur(img, (5,5), 0)

sobelx = cv.Sobel(blur, cv.CV_64F, 1,0, ksize = 1)
sobely = cv.Sobel(blur, cv.CV_64F, 0,1, ksize = 1)

blur_grad_mag_L2 = cv.magnitude(sobelx,sobely)
blur_grad_mag_L2 = cv.convertScaleAbs(blur_grad_mag_L2)

cv.imshow("result", np.hstack((grad_mag_L2,blur_grad_mag_L2)))
cv.waitKey(0)
cv.destroyAllWindows()

The noises become less obvious after gaussian blur is applied. 

In [5]:
blur = cv.GaussianBlur(img, (5,5), 0)
laplacian = cv.Laplacian(blur,cv.CV_64F, ksize =3)

laplacian_8u = cv.convertScaleAbs(laplacian)

cv.imshow("result", np.hstack((grad_mag_L2, blur_grad_mag_L2, laplacian_8u)))
cv.waitKey(0)
cv.destroyAllWindows()

The most optimal image processing pathway is apply gaussian filter, then apply sobel edge detection as the image processed is obvious with the edge and the noise produced is the least among three of them.

# Question 3

In [6]:
img = cv.imread('pineapple.jfif', 0)
img_blur = cv.GaussianBlur(img, (5,5), 0)

# Sobel
sobelx = cv.Sobel(img_blur, cv.CV_64F, 1,0, ksize = 3)
sobely = cv.Sobel(img_blur, cv.CV_64F, 0,1, ksize = 3)
blur_grad_mag_L2 = cv.magnitude(sobelx,sobely)
blur_grad_mag_L2 = cv.convertScaleAbs(blur_grad_mag_L2)

#laplacian
laplacian_blur = cv.Laplacian(img_blur, cv.CV_64F, ksize = 3)
laplacian_blur = np.uint8(np.absolute(laplacian_blur))

#Prewitt
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
img_prewittx = cv.filter2D(img_blur, -1, kernelx)
img_prewitty = cv.filter2D(img_blur, -1, kernely)
img_comb_prewitt = (img_prewittx + img_prewitty)

#Scharr Derivatives
scharr_X = cv.Scharr(img_blur, cv.CV_64F, 1, 0) 
scharr_X_abs = np.uint8(np.absolute(scharr_X)) 
scharr_Y = cv.Scharr(img_blur, cv.CV_64F, 0, 1) 
scharr_Y_abs = np.uint8(np.absolute(scharr_Y)) 
scharr_XY_combined = cv.bitwise_or(scharr_Y_abs,scharr_X_abs) 

#Canny operators
edges = cv.Canny(img_blur, 100, 200)

# all output
cv.imshow("result", np.hstack((blur_grad_mag_L2,laplacian_blur, img_comb_prewitt, scharr_XY_combined, edges)))
cv.waitKey(0)
cv.destroyAllWindows()

- Sobel: The edge detected is spreaded out.
- Laplacian: Can clearly see the edge but its too small.
- Prewitt: The edge detected is blurry but can see the shape of pineapple.
- Scharr Derivatives: All the edges in image is detected and its quite complicated.
- Canny operators: The cleanest edge detected.

# Question 4

In [43]:
img = cv.imread('electronic.jfif')
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 

img = cv.GaussianBlur(img_gray, (5,5), 0)
ret, thresh = cv.threshold(img_gray, 180, 255, cv.THRESH_BINARY)

contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

for i in contours:
    area = cv.contourArea(i)
    if area >= 1000:
        cnt = i
        x, y, w, h = cv.boundingRect(cnt)
        cv.rectangle(img_gray, (x,y), (x+w, y+h), (0, 255, 255), 2)
    else:
        continue

cv.imshow('result', img)
cv.waitKey(0)
cv.destroyAllWindows()